## Find extent of Flooded areas as modeled for Disaster Risk Profile by Island Region in The Bahamas

In [ ]:
import os
import rasterio as rio
import pandas as pd
import geopandas as gpd
import rasterstats as rs
import plotly.express as px

In [ ]:
def zonal_stats(geodf, raster_path, prefix, stats, grpby):
    """

    :param geodf:  Geodataframe with polygons with areas to be summarized
    :param raster_path:   path to raster
    :param prefix: prefix to indicate data source 
    :param stat: statistics to calculate - see https://pythonhosted.org/rasterstats/manual.html#zonal-statistics for options
    :return:  data frame with calculated stats
    """
    zstats = rs.zonal_stats(geodf, raster_path,
                           geojson_out=True, prefix=prefix,
                          stats=stats,
                          all_touched=True)
    statsdf = gpd.GeoDataFrame.from_features(zstats)
    stats_columns = [prefix + stat for stat in stats]
    keep_columns = [grpby] + stats_columns
    summary = statsdf[keep_columns]
#     summary = statsdf[keep_columns].groupby([grpby]).sum()  # I there are > 1 record for the summary columns
#     summary.reset_index(inplace=True)
    return summary

In [ ]:
# Variables for directories, files, and layers
raster_dir = '/Users/arbailey/Google Drive/Shared drives/brigBH2021/data/work/risks/flood_present'
flood_rasters = {
    'rp005':'A1_TWL_RP5_32618.tif',
    'rp010':'A1_TWL_RP10_32618.tif',
    'rp025':'A1_TWL_RP25_32618.tif',
    'rp050':'A1_TWL_RP50_32618.tif',
    'rp100':'A1_TWL_RP100_32618.tif'
}
bounds_dir = '/Users/arbailey/Google Drive/Shared drives/brigBH2021/data/work/bounds'
bounds_gpkg = os.path.join(bounds_dir,'bhs_bounds.gpkg')
islands_layer = 'LandPoly_Bahamas_subregions_32618'
islands_2kmbuffer_layer = 'bhs_land_inside2kmbuffer_32618'

In [ ]:
# Load vector layers with regions to summarize rasters
# Full island land 
islands_gdf = gpd.read_file(bounds_gpkg, layer=islands_layer)
print(islands_gdf.head())
# Island area within 2km of shoreline
islands_2kmbuffer_gdf = gpd.read_file(bounds_gpkg, layer=islands_2kmbuffer_layer)

In [ ]:
# get raster resolution from one of the flood rasters
rast = rio.open(os.path.join(raster_dir, flood_rasters['rp005']))
pixel_resolution = rast.res[0]
pixel_area_m2 = pixel_resolution ** 2
pixel_area_m2

In [ ]:
# Example -- Return Period 5 years
# rp05_raster = os.path.join(raster_dir, flood_rasters['rp05'])
# print("Zonal Stats with ", rp05_raster)
# rp05_count = zonal_stats(islands_gdf, rp05_raster, 'rp05_', 'count', 'Island')
# print(rp05_count)
# rp05_max = zonal_stats(islands_gdf, rp05_raster, 'rp05_', 'max', 'Island')
# print(rp05_max)

In [ ]:
###. WIDE DAta Frame
# Loop through all rasters and metrics to create a zonal stats summary of each
# Add each data frame to a list  -- This is for a WIDE data frame
summary_column = 'Island'
results = []
results_buffer = []
for rp, tif in flood_rasters.items():
    print(rp, tif)
    raster_path = os.path.join(raster_dir, tif)
    # Summarize stats for full island area
    prefix = rp # + "_"
    metrics = ['count','max']
    summary_df = zonal_stats(islands_gdf, raster_path, prefix, metrics, summary_column)
    results.append(summary_df)
    # Summarize results for 2km buffer coastline strip
    prefix = rp + "buff"
    summary_df = zonal_stats(islands_2kmbuffer_gdf, raster_path, prefix, ['count'], summary_column)
    results_buffer.append(summary_df)    
#     print(summary_df) 

# Join all the data frames together on the summary column
# This makes a WIDE data frame
for i, df in enumerate(results + results_buffer):
#     print(i, df)
    if i == 0:
        joined_df = df
    else:
        joined_df = pd.merge(left=joined_df, right=df, how='left', left_on=summary_column, right_on=summary_column)
joined_df 

In [ ]:

## LONG Data Frame
# Loop through all rasters and metrics to create a zonal stats summary of each
# Add each data frame to a list  -- This is for a LONG data frame
summary_column = 'Island'
results = []
results_buffer = []
for rp, tif in flood_rasters.items():
    print(rp, tif)
    raster_path = os.path.join(raster_dir, tif)
    # Summarize stats for full island area
    prefix = ""
    metrics = ['count','max']
    islands_df = zonal_stats(islands_gdf, raster_path, prefix, metrics, summary_column)
    islands_df['rp']=rp
    islands_df['buffer']='n'
    islands_df['floodarea_m2'] = islands_df['count'] * pixel_area_m2
    # Summarize results for 2km buffer coastline strip
    buffer_df = zonal_stats(islands_2kmbuffer_gdf, raster_path, prefix, ['count'], summary_column)
    buffer_df['rp']=rp
    buffer_df['buffer']='y'
    buffer_df['floodarea_buff2k_m2'] = buffer_df['count'] * pixel_area_m2
    
    islands_df['count_buff2k'] = buffer_df['count']
    islands_df['floodarea_buff2k_m2'] = buffer_df['floodarea_buff2k_m2']

    
    results.append(islands_df)
    results_buffer.append(buffer_df)    
#     print(summary_df) 

# Join all the data frames together on the summary column
# This makes a LONG data frame

stacked_df = pd.concat(results,axis=0, ignore_index=True)
print(stacked_df.head())

stacked_buffer_df = pd.concat(results_buffer,axis=0, ignore_index=True)
print(stacked_buffer_df.head())

In [ ]:
#  Add the total area of the island to the data frame
floodareas_df = pd.merge(left=stacked_df, right=islands_gdf, how='left', left_on=summary_column, right_on=summary_column)
floodareas_df['prop_areaflooded'] = floodareas_df['floodarea_m2'] / floodareas_df['area_m2']
floodareas_df['prop_buff2kflooded'] = floodareas_df['floodarea_buff2k_m2'] / floodareas_df['floodarea_m2']
floodareas_df

In [ ]:
# Visualize proportion of island area flooded for the different return periods.   Facets ordered by Island Name
fig = px.bar(floodareas_df.sort_values(by=['Island','rp']), x='rp',y='prop_areaflooded', facet_col='Island', facet_col_wrap=4, height=1200)
# fig.update_yaxes(matches=None)  # needed this before normalizing to proportion of area flooded
fig.show()

In [ ]:
# Visualize proportion of island area flooded for the different return periods.   Facets ordered by Island Total land area - smallest to largest

fig = px.bar(floodareas_df.sort_values(by=['area_m2','rp']), x='rp',y='prop_areaflooded', facet_col='Island', facet_col_wrap=4, height=1200)
# fig.update_yaxes(matches=None)  # needed this before normalizing to proportion of area flooded
fig.show()

In [ ]:
# Visualize proportion of 2km buffer area flooded for the different return periods.   Facets ordered by Island Total land area - smallest to largest

fig = px.bar(floodareas_df.sort_values(by=['area_m2','rp']), x='rp',y='prop_buff2kflooded', facet_col='Island', facet_col_wrap=4, height=1200)
# fig.update_yaxes(matches=None)  # needed this before normalizing to proportion of area flooded
fig.show()

In [ ]:
fig = px.bar(floodareas_df.sort_values(by=['area_m2','rp']), x='rp',y='prop_areaflooded', color='Island', barmode='group', height=1200)
fig.show()
